# Creating and Inserting Data

## Setup

In [ ]:
import os
import tempfile
from pathlib import Path
from urllib.request import urlretrieve

tempdir = Path(tempfile.gettempdir())
geography_db_file = tempdir / 'geography.db'

if geography_db_file.exists():
    geography_db_file.unlink()

_ = urlretrieve(
    'https://storage.googleapis.com/ibis-tutorial-data/geography.db',
    geography_db_file,
)

In [ ]:
import ibis

ibis.options.interactive = True

connection = ibis.sqlite.connect(geography_db_file)

## Creating new tables from Ibis expressions


Suppose you have an Ibis expression that produces a table:

In [ ]:
countries = connection.table('countries')

continent_name = (
    countries.continent.case()
    .when('AF', 'Africa')
    .when('AN', 'Antarctica')
    .when('AS', 'Asia')
    .when('EU', 'Europe')
    .when('NA', 'North America')
    .when('OC', 'Oceania')
    .when('SA', 'South America')
    .else_(countries.continent)
    .end()
    .name('continent_name')
)

expr = countries[countries.continent, continent_name].distinct()
expr

To create a table in the database from the results of this expression, use the connection's `create_table` method:

In [ ]:
connection.create_table('continents', expr)

In [ ]:
continents = connection.table('continents')
continents

Tables can be similarly dropped with `drop_table`

In [ ]:
connection.drop_table('continents')

## Creating new tables from in-memory Pandas dataframes

Pandas and NumPy are convenient to create test data in memory as a dataframe. This can then be turned into an Ibis expression using `ibis.memtable`.

In [ ]:
import pandas as pd
import numpy as np


def make_students_df(num_records, random_seed=None):
    rng = np.random.default_rng(random_seed)
    return pd.DataFrame(
        {
            "firstname": rng.choice(["Alice", "Bob", "Jane", "John"], size=num_records),
            "birth_date": (
                pd.to_datetime("2021-01-01")
                + pd.to_timedelta(rng.integers(0, 365, size=num_records), unit="D")
            ),
            "math_grade": rng.normal(55, 10, size=num_records).clip(0, 100).round(1),
        }
    )

students_df = make_students_df(21, random_seed=42)
students_memtable = ibis.memtable(students_df)
students_memtable


By default `ibis.memtable` uses the `duckdb` in-memory backend to execute queries against the Pandas dataframe data efficiently.

We can then materialize it as a physical table for a specific backend if necessary:

In [ ]:
students_db_file = tempdir / "ibis_tutorial_students.duckdb"
if students_db_file.exists():
    students_db_file.unlink()

connection = ibis.duckdb.connect(students_db_file)
connection.create_table('students', students_memtable)
students = connection.table('students')
students.group_by(students.birth_date.month()).aggregate(
    count=students.count(),
    avg_math_grade=students.math_grade.mean(),
)

Note that NumPy, Pandas and `ibis.memtable` are only suitable to generate data that fits in memory. To generate data larger than memory, we can generate data in chunks and iteratively insert the chunks using `connection.insert(tablename, pandas_dataframe)`:

In [ ]:
connection.insert(students.get_name(), make_students_df(10_000, random_seed=43))
students.count()

In [ ]:
connection.insert(students.get_name(), make_students_df(10_000, random_seed=44))
students.count()

In [ ]:
students.group_by(students.birth_date.month()).aggregate(
    count=students.count(),
    avg_math_grade=students.math_grade.mean(),
)

## Inserting data into existing tables


Some backends support inserting data into existing tables from expressions. This can be done using `connection.insert('table_name', expr)`.